In [2]:
import requests
from data import DECLARATIONS_DIR

True

In [18]:
declaration_files = DECLARATIONS_DIR.glob("declarations/*.json")

import json

declarations = []

for d in DECLARATIONS_DIR.glob("declarations/*.json"):
    with open(d, "r") as f:
        declarations.append(json.load(f))


# Fix faulty twitter api json
for declaration in declarations:
    if declaration.get("name") == "X_Twitter-API-V1":
        extra_keys = set(declaration.keys()) - {"name", "documents"}
        for key in extra_keys:
            corrected_key = "Developer Utilities" if key == "Developper Utilities" else key
            declaration["documents"][corrected_key] = declaration.pop(key)

In [102]:
from uuid import uuid4

def process_declaration(declaration):
    res = []
    name = declaration["name"]
    for k, v in declaration["documents"].items():
        if "combine" in v:
            doc_sources = v["combine"]
        else:
            doc_sources = [v]
            
        for source in doc_sources:
            res.append({"name": name, "document": k, "url": source["fetch"]})
    return res

sources = []
for d in declarations:
    sources.extend(process_declaration(d))


for i, source in enumerate(sources):
    source["id"] = i

print("num sources:", len(sources))
print(sources[500])

num sources: 1085
{'name': 'Facebook_Graph-API', 'document': 'Canvas Endpoint', 'url': 'https://developers.facebook.com/docs/graph-api/reference/canvas-footer/', 'id': 500}


# Postlight Scraper
Postlight is an opensource parser that extract important content from a URL. See https://github.com/postlight/parser 

We host postlight locally and scrape all URLs from the declarations repo


1. Install npm and yarn: https://classic.yarnpkg.com/lang/en/docs/install/#debian-stable
2. Run the postlight parser API
```
git clone https://github.com/postlight/parser-api.git
cd parser-api
yarn install
yarn serve
```

In [80]:
from typing import Dict

import requests

POSTLIGHT_API_URL = "http://localhost:3000"

def scrape_with_postlight(url: str) -> Dict[str, str]:
    response = requests.get(POSTLIGHT_API_URL + "/parser", params={"url": url})
    return response.json()


In [105]:
from tqdm.notebook import tqdm
import time

scraped = {}

for source in tqdm(sources):
    time.sleep(0.1)
    try: 
        s = scrape_with_postlight(source["url"])
        scraped[source["id"]] = s
    except Exception:
        print(f"could not parse {source['url']}")

  0%|          | 0/1085 [00:00<?, ?it/s]

could not parse https://storage.googleapis.com/transparencyreport/report-downloads/pdf-report-27_2023-8-28_2023-9-10_en_v1.pdf
could not parse https://r-xx.bstatic.com/data/mobile/dsa_transparency_report_bf3fdc24.pdf
could not parse https://www.tiktok.com/transparency/fr-fr/government-removal-requests-2022-2/
could not parse https://assets.aboutamazon.com/cd/28/4d02dd2e41ec8c6d1bc341e9d919/amazon-eu-store-transparency-report-jan-june-2023.pdf
could not parse https://developer.apple.com/support/downloads/Ad-Repository.pdf


In [126]:
# Short documents probably didnt work...

sum([len(s["content"])<100 for s in scraped.values()])

for s in scraped.values():
    if len(s["content"]) < 100:
        print(s["url"], s["content"])

https://developers.facebook.com/docs/features-reference/ad-targeting-data-access/ <div class="_4lr1" id="documentation_primary_nav_pagelet"></div>
https://developers.facebook.com/docs/features-reference/ads-management-standard-access/ <div class="_4lr1" id="documentation_primary_nav_pagelet"></div>
https://developers.facebook.com/docs/features-reference/business-asset-user-profile-access/ <div class="_4lr1" id="documentation_primary_nav_pagelet"></div>
https://developers.facebook.com/docs/features-reference/groups-api/ <div class="_4lr1" id="documentation_primary_nav_pagelet"></div>
https://developers.facebook.com/docs/features-reference/human-agent/ <div class="_4lr1" id="documentation_primary_nav_pagelet"></div>
https://developers.facebook.com/docs/features-reference/instagram-public-content-access/ <div class="_4lr1" id="documentation_primary_nav_pagelet"></div>
https://developers.facebook.com/docs/features-reference/live-video-api/ <div class="_4lr1" id="documentation_primary_nav_p

In [127]:
!pip install pypandoc


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [139]:
from bs4 import BeautifulSoup

def html_to_plaintext(html: str) -> str:
    soup = BeautifulSoup(html, features="lxml")
    return "\n\n".join([p.get_text() for p in soup.find_all("p")])
    
def convert_html_to_markdown(html: str) -> str:
    # Convert HTML to Markdown using Pandoc
    output = pypandoc.convert_text(html, 'md', format='html')
    return output


In [143]:
print(convert_html_to_markdown(scraped[900]["content"]))

::: is-table-default
If you have been working with the v1.1
[statuses/sample](/content/developer-twitter/en/docs/twitter-api/v1/tweets/sample-realtime/overview/get_statuses_sample)
endpoint, the goal of this guide is to help you understand the
similarities and differences between the standard and Twitter API v2
sample stream endpoints.

-   **Differences**
    -   Endpoint URLs
    -   App and Project requirements
    -   Authentication method
    -   Response data format
    -   Request parameters
    -   Availability of recovery and redundancy features

### Differences 

#### Endpoint URLs

-   Standard v1.1 endpoints:
    -   [https://stream.twitter.com/1.1/statuses/sample]{.code-inline}
-   Twitter API v2 endpoint:
    -   [https://api.twitter.com/2/tweets/sample/stream]{.code-inline}

#### App and Project requirements

The Twitter API v2 endpoints require that you use credentials from
a [developer
App](https://aem-staging.twitter.biz/content/developer-twitter/en/docs/apps.html) th

In [144]:
from IPython.display import HTML

display(HTML(scraped[900]["content"]))

Standard v1.1 parameter,Details
Delimited,"With the v1.1 endpoint, setting this to the string length indicates that statuses should be delimited in the stream, so that clients know how many bytes to read before the end of the status message. This functionality is not available with Twitter API v2"
Stall_warnings,"With the v1.1 endpoint, setting this parameter to the string true will cause periodic messages to be delivered if the client is in danger of being disconnected. With Twitter API v2, stall warnings are sent by default with the new line sent every so often."
